In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url='https://chipublib.bibliocommons.com/events/rss/all'
program_list={}
my_dict = {"Title":[],"Link":[],"Start Date":[]}

In [3]:
r=requests.get(url)
soup=BeautifulSoup(r.content, features='xml')
#         return print('CPL scrape successful: ',r.status_code)
programs = soup.findAll('item')

In [4]:
for p in programs:
            title=p.find('title').text
            link=p.find('link').text
            start_date=p.find('bc:start_date_local').text
            my_dict["Title"].append(title)
            my_dict["Link"].append(link)
            my_dict["Start Date"].append(start_date)
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)

In [5]:
type(my_dict)

dict

In [12]:
my_dict

{'Title': ['Take and Make Kit: Decorative Clay Dish',
  'Cantos y Cuentos Bilingual Story Time',
  'Story Time',
  'Stories and Songs',
  'Mother Goose on the Loose',
  'Family Story Time in The Not-So-Secret Garden',
  'Preschool Story Time',
  'Preschool Story Time',
  'Preschool Story Time',
  'Tiny Tykes Story Time',
  'Drawing Prompt Wednesdays',
  'Cantos y Cuentos ',
  'Preschool Story Time',
  'Family Story Time in The Not-So-Secret Garden',
  'Grab and Go: Paper Rose Craft',
  'Teen Grab and Go Kits at Sherman Park Branch',
  'Neighborhood Tour: Coffee and Conversation: Chicago’s Diners ',
  ' Grab and Go: PPE Wellness Kit ',
  'Little Learners Story Time',
  'Stories and Songs',
  'Learning Circle: The Art of Storytelling with Pixar',
  'After School Meals ',
  'After School Meals ',
  'Family Game Night',
  'Baby Dance',
  'Anime and Manga Club',
  'Reading Came First Book Club: Wild Seed by Octavia Butler',
  'After School Meals ',
  '3d Prints: Tinkercad Tutorials',
  'BAW

In [13]:
df = pd.DataFrame(my_dict)

In [14]:
df

,Title,Link,Start Date
0,Take and Make Kit: Decorative Clay Dish,https://chipublib.bibliocommons.com/events/615...,2021-10-06T09:30
1,Cantos y Cuentos Bilingual Story Time,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:00
2,Story Time,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:00
3,Stories and Songs,https://chipublib.bibliocommons.com/events/613...,2021-10-06T10:30
4,Mother Goose on the Loose,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:30
...,...,...,...
1366,Books 'n Babes,https://chipublib.bibliocommons.com/events/615...,2022-02-28T10:30
1367,Books 'n Babes,https://chipublib.bibliocommons.com/events/615...,2022-03-14T10:30
1368,Books 'n Babes,https://chipublib.bibliocommons.com/events/615...,2022-03-28T10:30
1369,"Law at the Library: Wills, Trusts and Estate P...",https://chipublib.bibliocommons.com/events/611...,2022-03-29T18:00


In [7]:
# def cpl_programs(url):
#     program_list=[]
#     try:
#         r=requests.get(url)
#         soup=BeautifulSoup(r.content, features='xml')
# #         return print('CPL scrape successful: ',r.status_code)
#         programs = soup.findAll('item')
    
#         for p in programs:
#             title=p.find('title').text
#             link=p.find('link').text
#             start_date=p.find('bc:start_date_local').text
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)
        

#         return print(program_list)
#     except Exception as e:
#         print('FAILED. See exception: ')
#         print(e)
        


In [8]:
# print('Starting scrape')
# cpl_programs(url)
# print('Finished')

In [9]:
# test=cpl_programs(url)